# Import Packages

In [1]:
import pandas as pd
import string
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import SnowballStemmer 
from nltk.stem import WordNetLemmatizer 

nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/kaitlynzeichick/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/kaitlynzeichick/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Load and Merge Spreadsheets

In [272]:
demographic = pd.read_csv('demographic.txt')

In [273]:
happiness = pd.read_csv('cleaned_hm.txt')

In [274]:
df = pd.merge(demographic, happiness, left_on='wid', right_on='wid', how='left')

In [275]:
pd.set_option("display.max_colwidth", -1)

In [276]:
df.head()

,wid,age,country,gender,marital,parenthood,hmid,reflection_period,original_hm,cleaned_hm,modified,num_sentence,ground_truth_category,predicted_category
0,1,37.0,USA,m,married,y,27854.0,24h,My wife and i are celebrating our ten year anniversary today.,My wife and i are celebrating our ten year anniversary today.,True,1.0,NaN,affection
1,1,37.0,USA,m,married,y,28054.0,24h,My mother called out of the blue to tell me how proud she is of me.,My mother called out of the blue to tell me how proud she is of me.,True,1.0,affection,affection
2,1,37.0,USA,m,married,y,28254.0,24h,Today i took the day off from my part time job to go and have a brunch date with my wonderful wife.,Today i took the day off from my part time job to go and have a brunch date with my wonderful wife.,True,1.0,NaN,affection
3,1,37.0,USA,m,married,y,28489.0,24h,I just got a $21 bonus on a mturk task!,I just got a $21 bonus on a mturk task!,True,1.0,NaN,achievement
4,1,37.0,USA,m,married,y,28991.0,24h,My wife cooked me a surprise dinner to take to work with me.,My wife cooked me a surprise dinner to take to work with me.,True,1.0,NaN,affection


# Clean Text

## Removing Columns

In [277]:
df.drop(columns=['original_hm', 'modified', 'num_sentence', 'ground_truth_category', 
                 'predicted_category', 'hmid'], inplace = True)

In [278]:
df.rename(columns={'cleaned_hm': 'text'}, inplace = True)

In [279]:
df.head()

,wid,age,country,gender,marital,parenthood,reflection_period,text
0,1,37.0,USA,m,married,y,24h,My wife and i are celebrating our ten year anniversary today.
1,1,37.0,USA,m,married,y,24h,My mother called out of the blue to tell me how proud she is of me.
2,1,37.0,USA,m,married,y,24h,Today i took the day off from my part time job to go and have a brunch date with my wonderful wife.
3,1,37.0,USA,m,married,y,24h,I just got a $21 bonus on a mturk task!
4,1,37.0,USA,m,married,y,24h,My wife cooked me a surprise dinner to take to work with me.


## Null Values

In [280]:
df.dropna(how = 'any', inplace = True)

In [281]:
df.shape

(100082, 8)

## Punctuation

In [282]:
df['text_cleaned'] = df['text']

In [283]:
def remove_punctuation(row):
    """
    Remove all punctuation from the text.
    """
    text = row['text_cleaned']
    return text.translate(str.maketrans('', '', string.punctuation))
    
df['text_cleaned'] = df.apply(lambda row: remove_punctuation(row), axis=1)

## Words with Numbers

In [285]:
def remove_numbers(row):
    """
    Remove all words that include numbers.
    """
    list_words = row['text_cleaned'].split()
    sentence = row['text_cleaned'].split()
    for word in sentence:
        for character in word:
            if character.isdigit():
                try:
                    list_words.remove(word)
                except ValueError:
                    pass
    final_words = ' '.join(list_words)
    return final_words


df['text_cleaned'] = df.apply(lambda row: remove_numbers(row), axis=1)

In [286]:
df.head()

,wid,age,country,gender,marital,parenthood,reflection_period,text,text_cleaned
0,1,37.0,USA,m,married,y,24h,My wife and i are celebrating our ten year anniversary today.,My wife and i are celebrating our ten year anniversary today
1,1,37.0,USA,m,married,y,24h,My mother called out of the blue to tell me how proud she is of me.,My mother called out of the blue to tell me how proud she is of me
2,1,37.0,USA,m,married,y,24h,Today i took the day off from my part time job to go and have a brunch date with my wonderful wife.,Today i took the day off from my part time job to go and have a brunch date with my wonderful wife
3,1,37.0,USA,m,married,y,24h,I just got a $21 bonus on a mturk task!,I just got a bonus on a mturk task
4,1,37.0,USA,m,married,y,24h,My wife cooked me a surprise dinner to take to work with me.,My wife cooked me a surprise dinner to take to work with me


## Lower Case

In [287]:
df['text_cleaned'] = df['text_cleaned'].str.lower()

In [288]:
df.head()

,wid,age,country,gender,marital,parenthood,reflection_period,text,text_cleaned
0,1,37.0,USA,m,married,y,24h,My wife and i are celebrating our ten year anniversary today.,my wife and i are celebrating our ten year anniversary today
1,1,37.0,USA,m,married,y,24h,My mother called out of the blue to tell me how proud she is of me.,my mother called out of the blue to tell me how proud she is of me
2,1,37.0,USA,m,married,y,24h,Today i took the day off from my part time job to go and have a brunch date with my wonderful wife.,today i took the day off from my part time job to go and have a brunch date with my wonderful wife
3,1,37.0,USA,m,married,y,24h,I just got a $21 bonus on a mturk task!,i just got a bonus on a mturk task
4,1,37.0,USA,m,married,y,24h,My wife cooked me a surprise dinner to take to work with me.,my wife cooked me a surprise dinner to take to work with me


## Tokenize

In [289]:
df['text_cleaned'] = df.apply(lambda row: nltk.word_tokenize(row['text_cleaned']), axis=1)

In [290]:
df.head()

,wid,age,country,gender,marital,parenthood,reflection_period,text,text_cleaned
0,1,37.0,USA,m,married,y,24h,My wife and i are celebrating our ten year anniversary today.,"[my, wife, and, i, are, celebrating, our, ten, year, anniversary, today]"
1,1,37.0,USA,m,married,y,24h,My mother called out of the blue to tell me how proud she is of me.,"[my, mother, called, out, of, the, blue, to, tell, me, how, proud, she, is, of, me]"
2,1,37.0,USA,m,married,y,24h,Today i took the day off from my part time job to go and have a brunch date with my wonderful wife.,"[today, i, took, the, day, off, from, my, part, time, job, to, go, and, have, a, brunch, date, with, my, wonderful, wife]"
3,1,37.0,USA,m,married,y,24h,I just got a $21 bonus on a mturk task!,"[i, just, got, a, bonus, on, a, mturk, task]"
4,1,37.0,USA,m,married,y,24h,My wife cooked me a surprise dinner to take to work with me.,"[my, wife, cooked, me, a, surprise, dinner, to, take, to, work, with, me]"


# Processing

## Stemming

In [ ]:
stemmer = SnowballStemmer("english")   

In [ ]:
def stem(row):
    stem_list = []
    words = row['text_cleaned']
    for word in words:
        stem_list.append(stemmer.stem(word))
    return stem_list

df['text_cleaned'] = df.apply(lambda row: stem(row), axis=1)

## Lemmatize

In [291]:
lemmatizer = WordNetLemmatizer() 

In [292]:
def lemmatize(row):
    lemm_list = []
    words = row['text_cleaned']
    for word in words:
        lemm_list.append(lemmatizer.lemmatize(word))
    return lemm_list

df['text_cleaned'] = df.apply(lambda row: lemmatize(row), axis=1)

In [293]:
df.head()

,wid,age,country,gender,marital,parenthood,reflection_period,text,text_cleaned
0,1,37.0,USA,m,married,y,24h,My wife and i are celebrating our ten year anniversary today.,"[my, wife, and, i, are, celebrating, our, ten, year, anniversary, today]"
1,1,37.0,USA,m,married,y,24h,My mother called out of the blue to tell me how proud she is of me.,"[my, mother, called, out, of, the, blue, to, tell, me, how, proud, she, is, of, me]"
2,1,37.0,USA,m,married,y,24h,Today i took the day off from my part time job to go and have a brunch date with my wonderful wife.,"[today, i, took, the, day, off, from, my, part, time, job, to, go, and, have, a, brunch, date, with, my, wonderful, wife]"
3,1,37.0,USA,m,married,y,24h,I just got a $21 bonus on a mturk task!,"[i, just, got, a, bonus, on, a, mturk, task]"
4,1,37.0,USA,m,married,y,24h,My wife cooked me a surprise dinner to take to work with me.,"[my, wife, cooked, me, a, surprise, dinner, to, take, to, work, with, me]"


## Stop Words

In [294]:
from sklearn.feature_extraction import text 

In [297]:
specific_stop_words = ['good', 'great', 'really', 'happy', 'felt', 'feel', 'happiness',
                        'happiest']

# Add specific stop words to general english stop words
stop_words = text.ENGLISH_STOP_WORDS.union(specific_stop_words)

In [298]:
df['text_cleaned'] = df['text_cleaned'].apply(lambda x: [item for item in x if item not in stop_words])

In [299]:
df.head()

,wid,age,country,gender,marital,parenthood,reflection_period,text,text_cleaned
0,1,37.0,USA,m,married,y,24h,My wife and i are celebrating our ten year anniversary today.,"[wife, celebrating, year, anniversary, today]"
1,1,37.0,USA,m,married,y,24h,My mother called out of the blue to tell me how proud she is of me.,"[mother, called, blue, tell, proud]"
2,1,37.0,USA,m,married,y,24h,Today i took the day off from my part time job to go and have a brunch date with my wonderful wife.,"[today, took, day, time, job, brunch, date, wonderful, wife]"
3,1,37.0,USA,m,married,y,24h,I just got a $21 bonus on a mturk task!,"[just, got, bonus, mturk, task]"
4,1,37.0,USA,m,married,y,24h,My wife cooked me a surprise dinner to take to work with me.,"[wife, cooked, surprise, dinner, work]"


## Linguistic Annotations

In [112]:
import spacy

/Users/kaitlynzeichick/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [115]:
nlp = spacy.load("en_core_web_sm")

In [126]:
# For testing purposes
df_test = df.head(10).copy(deep=True)

In [160]:
def print_speech(row):
    doc = nlp(row['text']) 
    root_verb = []
    for token in doc:
        print(token, token.pos_, token.dep_, token.tag_)
    
df_test['root_verb'] = df_test.apply(lambda row: print_speech(row), axis=1)

My PRON poss PRP$
wife NOUN nsubj NN
and CCONJ cc CC
i PRON conj PRP
are AUX aux VBP
celebrating VERB ROOT VBG
our PRON poss PRP$
ten NUM nummod CD
year NOUN compound NN
anniversary NOUN dobj NN
today NOUN npadvmod NN
. PUNCT punct .
My PRON poss PRP$
mother NOUN nsubj NN
called VERB ROOT VBD
out SCONJ prep IN
of ADP prep IN
the DET det DT
blue NOUN pobj NN
to PART aux TO
tell VERB advcl VB
me PRON dobj PRP
how ADV advmod WRB
proud ADJ acomp JJ
she PRON nsubj PRP
is AUX ccomp VBZ
of ADP prep IN
me PRON pobj PRP
. PUNCT punct .
Today NOUN npadvmod NN
i PRON nsubj PRP
took VERB ROOT VBD
the DET det DT
day NOUN dobj NN
off ADV advmod RB
from ADP prep IN
my PRON poss PRP$
part NOUN compound NN
time NOUN compound NN
job NOUN pobj NN
to PART aux TO
go VERB advcl VB
and CCONJ cc CC
have VERB conj VB
a DET det DT
brunch NOUN compound NN
date NOUN dobj NN
with ADP prep IN
my PRON poss PRP$
wonderful ADJ amod JJ
wife NOUN pobj NN
. PUNCT punct .
I PRON nsubj PRP
just ADV advmod RB
got VERB ROOT 

In [153]:
def find_verbs(row):
    doc = nlp(row['text']) 
    verbs = []
    for token in doc:
        if token.pos_ == 'VERB':
            verbs.append(token)
    return verbs
    
df['verbs'] = df.apply(lambda row: find_verbs(row), axis=1)

In [154]:
def find_nouns(row):
    doc = nlp(row['text']) 
    nouns = []
    for token in doc:
        if token.pos_ == 'NOUN':
            nouns.append(token)
    return nouns
    
df['nouns'] = df.apply(lambda row: find_nouns(row), axis=1)

In [155]:
def find_proto_agent(row):
    doc = nlp(row['text']) 
    nsubj = []
    for token in doc:
        if token.dep_ == 'nsubj':
            nsubj.append(token)
    return nsubj
    
df['proto_agent'] = df.apply(lambda row: find_proto_agent(row), axis=1)

In [156]:
def find_passive_agent(row):
    doc = nlp(row['text']) 
    nsubjpass = []
    for token in doc:
        if token.dep_ == 'nsubjpass':
            nsubjpass.append(token)
    return nsubjpass
    
df['passive_agent'] = df.apply(lambda row: find_passive_agent(row), axis=1)

In [157]:
def find_root_verb(row):
    doc = nlp(row['text']) 
    root_verb = []
    for token in doc:
        if token.dep_ == 'ROOT':
            root_verb.append(token)
    return root_verb
    
df['root_verb'] = df.apply(lambda row: find_root_verb(row), axis=1)

In [162]:
def find_direct_object(row):
    doc = nlp(row['text']) 
    dir_obj = []
    for token in doc:
        if token.dep_ == 'dobj':
            dir_obj.append(token)
    return dir_obj
    
df['direct_object'] = df.apply(lambda row: find_direct_object(row), axis=1)

In [163]:
df.head()

,wid,age,country,gender,marital,parenthood,reflection_period,text,text_cleaned,verbs,nouns,proto_agent,passive_agent,root_verb,direct_object
0,1,37.0,USA,m,married,y,24h,My wife and i are celebrating our ten year anniversary today.,"[wife, celebrating, year, anniversary, today]",[celebrating],"[wife, year, anniversary, today]",[wife],[],[celebrating],[anniversary]
1,1,37.0,USA,m,married,y,24h,My mother called out of the blue to tell me how proud she is of me.,"[mother, called, blue, tell, proud]","[called, tell]","[mother, blue]","[mother, she]",[],[called],[me]
2,1,37.0,USA,m,married,y,24h,Today i took the day off from my part time job to go and have a brunch date with my wonderful wife.,"[today, took, day, time, job, brunch, date, wonderful, wife]","[took, go, have]","[Today, day, part, time, job, brunch, date, wife]",[i],[],[took],"[day, date]"
3,1,37.0,USA,m,married,y,24h,I just got a $21 bonus on a mturk task!,"[just, got, bonus, mturk, task]",[got],"[bonus, mturk, task]",[I],[],[got],[bonus]
4,1,37.0,USA,m,married,y,24h,My wife cooked me a surprise dinner to take to work with me.,"[wife, cooked, surprise, dinner, work]","[cooked, take, work]","[wife, surprise, dinner]",[wife],[],[cooked],[dinner]
5,1,37.0,USA,m,married,y,24h,I bought my son a gift from the MTA store and left it on his bed so when he got home from school. He texted me a little while ago and told me he loved it.,"[bought, son, gift, mta, store, left, bed, got, home, school, texted, little, ago, told, loved]","[bought, left, got, texted, told, loved]","[son, gift, store, bed, school, while]","[I, he, He, he]",[],"[bought, texted]","[son, gift, it, me, me, it]"
6,1,37.0,USA,m,married,y,24h,The stock i started investing in went up slightly.,"[stock, started, investing, went, slightly]","[started, investing, went]",[stock],"[stock, i]",[],[went],[]
7,1,37.0,USA,m,married,y,24h,My oldest son got accepted into a great High School and is excited about it.,"[oldest, son, got, accepted, high, school, excited]","[accepted, is]",[son],[],[son],[accepted],[]
8,1,37.0,USA,m,married,y,24h,My wife surprised me and bought me a new I-phone 6 so i could upgrade from the 4.,"[wife, surprised, bought, new, iphone, upgrade]","[surprised, bought, upgrade]","[wife, I, phone]","[wife, i]",[],[surprised],"[me, phone]"
9,1,37.0,USA,m,married,y,24h,My parents told me they will be stopping by to celebrate my son's birthday with us this weekend.,"[parent, told, stopping, celebrate, son, birthday, u, weekend]","[told, stopping, celebrate]","[parents, son, birthday, weekend]","[parents, they]",[],[told],"[me, birthday]"


# Clean Demographic Information

## Age

In [3]:
df = pd.read_csv('cleaned_dataset.csv', dtype={"age": str, "country": str,
                                              "gender": str, "marital": str,
                                              "parenthood": str})

In [4]:
df.shape

(99872, 17)

In [5]:
def clean_age(row):
    if row['age'] == 'prefer not to say' or row['age'] == 'čá' or row['age'] == '60yrs':
        return np.NaN
    else:
        age = float(row['age'])
        age = int(age)
        if (age > 100) or (age < 18):
            return np.NaN
        return age
        
df['age'] = df.apply(lambda row: clean_age(row), axis=1)

In [6]:
df['age'].isna().sum()

0

In [7]:
df.dropna(subset=['age'], inplace=True)

In [8]:
df['age'].unique()

array([37, 29, 25, 32, 35, 34, 61, 27, 45, 30, 40, 31, 28, 36, 44, 23, 26,
       42, 41, 57, 24, 38, 43, 48, 53, 46, 54, 65, 59, 22, 47, 39, 21, 33,
       55, 69, 68, 20, 49, 56, 62, 70, 83, 74, 51, 19, 52, 66, 58, 18, 60,
       50, 72, 63, 67, 64, 77, 98, 73, 79, 80, 84, 88, 71, 75, 76, 95, 81,
       78])

In [9]:
df.shape

(99872, 17)

In [10]:
df

,Unnamed: 0,Unnamed: 0.1,wid,age,country,gender,marital,parenthood,reflection_period,text,text_cleaned,verbs,nouns,proto_agent,passive_agent,root_verb,direct_object
0,0,0,1,37,USA,m,married,y,24h,My wife and i are celebrating our ten year ann...,"['wife', 'celebrating', 'year', 'anniversary',...",[celebrating],"[wife, year, anniversary, today]",[wife],[],[celebrating],[anniversary]
1,1,1,1,37,USA,m,married,y,24h,My mother called out of the blue to tell me ho...,"['mother', 'called', 'blue', 'tell', 'proud']","[called, tell]","[mother, blue]","[mother, she]",[],[called],[me]
2,2,2,1,37,USA,m,married,y,24h,Today i took the day off from my part time job...,"['today', 'took', 'day', 'time', 'job', 'brunc...","[took, go, have]","[Today, day, part, time, job, brunch, date, wife]",[i],[],[took],"[day, date]"
3,3,3,1,37,USA,m,married,y,24h,I just got a $21 bonus on a mturk task!,"['just', 'got', 'bonus', 'mturk', 'task']",[got],"[bonus, mturk, task]",[I],[],[got],[bonus]
4,4,4,1,37,USA,m,married,y,24h,My wife cooked me a surprise dinner to take to...,"['wife', 'cooked', 'surprise', 'dinner', 'work']","[cooked, take, work]","[wife, surprise, dinner]",[wife],[],[cooked],[dinner]
5,5,5,1,37,USA,m,married,y,24h,I bought my son a gift from the MTA store and ...,"['bought', 'son', 'gift', 'mta', 'store', 'lef...","[bought, left, got, texted, told, loved]","[son, gift, store, bed, school, while]","[I, he, He, he]",[],"[bought, texted]","[son, gift, it, me, me, it]"
6,6,6,1,37,USA,m,married,y,24h,The stock i started investing in went up sligh...,"['stock', 'started', 'investing', 'went', 'sli...","[started, investing, went]",[stock],"[stock, i]",[],[went],[]
7,7,7,1,37,USA,m,married,y,24h,My oldest son got accepted into a great High S...,"['oldest', 'son', 'got', 'accepted', 'high', '...","[accepted, is]",[son],[],[son],[accepted],[]
8,8,8,1,37,USA,m,married,y,24h,My wife surprised me and bought me a new I-pho...,"['wife', 'surprised', 'bought', 'new', 'iphone...","[surprised, bought, upgrade]","[wife, I, phone]","[wife, i]",[],[surprised],"[me, phone]"
9,9,9,1,37,USA,m,married,y,24h,My parents told me they will be stopping by to...,"['parent', 'told', 'stopping', 'celebrate', 's...","[told, stopping, celebrate]","[parents, son, birthday, weekend]","[parents, they]",[],[told],"[me, birthday]"


## Parenthood

In [11]:
df["parenthood"] = df["parenthood"].astype('category')
df["parenthood_cat"] = df["parenthood"].cat.codes

## Gender

In [12]:
df["gender"] = df["gender"].astype('category')
df["gender_cat"] = df["gender"].cat.codes

## Marital Status

In [13]:
df["marital"] = df["marital"].astype('category')
df["marital_cat"] = df["marital"].cat.codes

In [14]:
df

,Unnamed: 0,Unnamed: 0.1,wid,age,country,gender,marital,parenthood,reflection_period,text,text_cleaned,verbs,nouns,proto_agent,passive_agent,root_verb,direct_object,parenthood_cat,gender_cat,marital_cat
0,0,0,1,37,USA,m,married,y,24h,My wife and i are celebrating our ten year ann...,"['wife', 'celebrating', 'year', 'anniversary',...",[celebrating],"[wife, year, anniversary, today]",[wife],[],[celebrating],[anniversary],1,1,1
1,1,1,1,37,USA,m,married,y,24h,My mother called out of the blue to tell me ho...,"['mother', 'called', 'blue', 'tell', 'proud']","[called, tell]","[mother, blue]","[mother, she]",[],[called],[me],1,1,1
2,2,2,1,37,USA,m,married,y,24h,Today i took the day off from my part time job...,"['today', 'took', 'day', 'time', 'job', 'brunc...","[took, go, have]","[Today, day, part, time, job, brunch, date, wife]",[i],[],[took],"[day, date]",1,1,1
3,3,3,1,37,USA,m,married,y,24h,I just got a $21 bonus on a mturk task!,"['just', 'got', 'bonus', 'mturk', 'task']",[got],"[bonus, mturk, task]",[I],[],[got],[bonus],1,1,1
4,4,4,1,37,USA,m,married,y,24h,My wife cooked me a surprise dinner to take to...,"['wife', 'cooked', 'surprise', 'dinner', 'work']","[cooked, take, work]","[wife, surprise, dinner]",[wife],[],[cooked],[dinner],1,1,1
5,5,5,1,37,USA,m,married,y,24h,I bought my son a gift from the MTA store and ...,"['bought', 'son', 'gift', 'mta', 'store', 'lef...","[bought, left, got, texted, told, loved]","[son, gift, store, bed, school, while]","[I, he, He, he]",[],"[bought, texted]","[son, gift, it, me, me, it]",1,1,1
6,6,6,1,37,USA,m,married,y,24h,The stock i started investing in went up sligh...,"['stock', 'started', 'investing', 'went', 'sli...","[started, investing, went]",[stock],"[stock, i]",[],[went],[],1,1,1
7,7,7,1,37,USA,m,married,y,24h,My oldest son got accepted into a great High S...,"['oldest', 'son', 'got', 'accepted', 'high', '...","[accepted, is]",[son],[],[son],[accepted],[],1,1,1
8,8,8,1,37,USA,m,married,y,24h,My wife surprised me and bought me a new I-pho...,"['wife', 'surprised', 'bought', 'new', 'iphone...","[surprised, bought, upgrade]","[wife, I, phone]","[wife, i]",[],[surprised],"[me, phone]",1,1,1
9,9,9,1,37,USA,m,married,y,24h,My parents told me they will be stopping by to...,"['parent', 'told', 'stopping', 'celebrate', 's...","[told, stopping, celebrate]","[parents, son, birthday, weekend]","[parents, they]",[],[told],"[me, birthday]",1,1,1


# Save CSV File

In [15]:
df.to_csv('cleaned_dataset.csv')

In [139]:
from spacy import displacy

about_interest_text =  df_test['text'][1]
about_interest_doc = nlp(about_interest_text)
displacy.render(about_interest_doc, style='dep', jupyter=True)